In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import pickle
import gzip
import pandas as pd
import numpy as np
# from hmmlearn import hmm
import warnings
warnings.filterwarnings('ignore')
matplotlib.rcParams.update({'font.size': 14})

In [ ]:
# def convertPklz(filename):
#     inF = gzip.open("%s.pklz"%filename, 'rb')
#     outF = open("%s.pickle"%filename, 'wb')
#     outF.write(inF.read())
#     inF.close()
#     outF.close()
#     df = pd.read_pickle("%s.pickle"%filename)
#     df.to_pickle("%s.pickle"%filename)
#     print "Convert %s.pklz to %s.pickle"%(filename,filename)
# convertPklz('obs_seq_list_lanechange')
# # convertPklz('obs_seq_list_lanekeep')

In [ ]:
# # Load Observatoin List
# with gzip.open('obs_seq_list_lanechange.pklz','rb') as f:
#     obs_seq_list_lc = pickle.load(f)
# with gzip.open('obs_seq_list_lanekeep.pklz','rb') as f:
#     obs_seq_list_lk = pickle.load(f)

In [ ]:
# Load observation sequence
obs_seq_list_lc = pd.read_pickle("data/obs_seq_list_lanechange.pickle")
obs_seq_list_lk = pd.read_pickle("data/obs_seq_list_lanekeep_new.pickle")

In [ ]:
# # Load the trained hmms
# # filename = 'hmms_4.pklz'
# filename = 'hmms_4_2.pklz'
# with gzip.open(filename,'rb') as f:
#     hmms = pickle.load(f)
hmms_lc = pd.read_pickle("data/hmm_lc_4.pickle")
hmms_lk = pd.read_pickle("data/hmm_lk_2.pickle")
hmms = {'lc': hmms_lc ,'lk': hmms_lk}

In [ ]:
def getLCLikelyhoodData(hmms,obs_seq,window_size):
    likelyhood_lc = list()
    likelyhood_lk = list()
    index_list = list()
    for i in range(window_size,len(obs_seq)):
        start_index = max(i - window_size,0)
        end_index = i
        index_list.append((i-60)*.1)
        lc = hmms['lc'].score(obs_seq[start_index:end_index])
        lk = hmms['lk'].score(obs_seq[start_index:end_index])
        likelyhood_lc.append(lc/abs(lc+lk))
        likelyhood_lk.append(lk/abs(lc+lk))
    return (index_list, likelyhood_lc, likelyhood_lk)

def getLCLikelyhoodFig(index_list, likelyhood_lc, likelyhood_lk):
    fig = plt.figure()
    plt.plot(index_list,1+np.array(likelyhood_lc),'r',label='Lane-changing')
    plt.plot(index_list,1+np.array(likelyhood_lk),'b',label='Lane-keeping')
    plt.ylim([0,1.0])
    plt.ylabel('Relative Likelyhood')
    plt.xlabel('Relative time to lane-change (sec)')
    plt.legend(loc='upper left',fontsize=14)
    return fig

def getLCDetectionTimeList(index_list,likelyhood_lc,threshold):
    bool_array = np.insert((np.array(likelyhood_lc) > threshold),0,False)
    diff_array = np.diff(bool_array.astype(float))
    index_array = np.array(index_list)
    detection_time_list = index_array[diff_array == 1.0]
    return detection_time_list

def getPerformanceTupleList(hmms,obs_seq_list,window_size):
    # Gather performance data
    data_tuple_list = list()
    for obs_seq in obs_seq_list:
        data_tuple_list.append(getLCLikelyhoodData(hmms,obs_seq,window_size))
    return data_tuple_list

In [ ]:
# %prun dd = getLCLikelyhoodData(hmms,obs_seq_list_lc[90],100)

In [ ]:
# Select observation sequency list used for performance evaluation
obs_seq_list_for_lc = obs_seq_list_lc[500:1000]
obs_seq_list_for_lk = [obs[0:101] for obs in obs_seq_list_lk[500:1000]]

def getDataTupleLists(window_size):
    # Get detection data for lc
    data_tuple_list_lc = getPerformanceTupleList(hmms,obs_seq_list_for_lc,window_size)
    # Get deteciton data for lk
    data_tuple_list_lk = getPerformanceTupleList(hmms,obs_seq_list_for_lk,window_size)
    return (data_tuple_list_lc,data_tuple_list_lk)

In [ ]:
print len(obs_seq_list_for_lc[0])
print len(obs_seq_list_for_lk[0])

In [ ]:
# Get data tuple for lc an lk of differenent window sizes
window_size_list = [5,10,15,20]
data_tuple_dict_winsize = dict()
for window_size in window_size_list:
    data_tuple_list_lc,data_tuple_list_lk = getDataTupleLists(window_size)
    data_tuple_dict_winsize[window_size] = (data_tuple_list_lc,data_tuple_list_lk)
    print("Window Size %s Done."%(window_size))

In [ ]:
def getDetectionTimeData(data_tuple_list, threshold):
# Process performance data
    detection_time_data = list()
    for data_tuple in data_tuple_list:
        detection_time_data.append(getLCDetectionTimeList(data_tuple[0],data_tuple[1],threshold))
    return detection_time_data

def getDetectionTimeArray(detection_time_data):
    # Get detection time
    max_det_time_list = list()
    for det_time_array in detection_time_data:
        det_time = 0.0
        if len(det_time_array) > 0:
            det_time = det_time_array.max()
        max_det_time_list.append(det_time)
    return np.array(max_det_time_list)

In [ ]:
# Example of detection performance measurment for one lane-change
(index_list, likelyhood_lc, likelyhood_lk) = getLCLikelyhoodData(hmms,obs_seq_list_lc[556],10)
print "Predicted at %s sec" %(getLCDetectionTimeList(index_list,likelyhood_lc,-0.4))
fig = getLCLikelyhoodFig(index_list, likelyhood_lc, likelyhood_lk)
fig.set_size_inches(10,4)
# fig.savefig('relative_likelyhood_traj.pdf')
print len(index_list)

In [ ]:
(index_list, likelyhood_lc, likelyhood_lk) = getLCLikelyhoodData(hmms,obs_seq_list_for_lk[4],10)
# print "Predicted at %s sec" %(getLCDetectionTimeList(index_list,likelyhood_lc,-0.4))
fig = getLCLikelyhoodFig(index_list, likelyhood_lc, likelyhood_lk)
fig.set_size_inches(10,4)
print len(index_list)
ax = fig.axes[0]
ax.set_xlabel('Relative time (sec)')
# fig.savefig('relative_likelyhood_traj_lk.pdf')

In [ ]:
# Plot detection time histogram
max_det_time_array = getDetectionTimeArray(getDetectionTimeData(data_tuple_dict_winsize[10][0],-0.4))
fig = plt.figure(figsize=(10,4))
plt.hist(max_det_time_array,bins=20)
plt.xlabel('Lane-change detection time (s)')
plt.ylabel('Number of times (Out of 500)')
print "mean: %s std: %s"%(max_det_time_array.mean(),max_det_time_array.std())
# fig.savefig('lc_detection_histo.pdf')

In [ ]:
data_tuple_list_lc,data_tuple_list_lk = data_tuple_dict_winsize[10]
false_times_array = getDetectionTimeData(data_tuple_list_lk,-0.4)
false_times_count = np.array([len(array) for array in false_times_array])
fig = plt.figure(figsize=(10,4))
plt.hist(false_times_count,bins=20)
plt.xlabel('Number of false detection per trajectory')
plt.ylabel('Number of times (Out of 500)')
# fig.savefig('lk_detection_histo.pdf')

In [ ]:
data_tuple_list_lc,data_tuple_list_lk = data_tuple_dict_winsize[10]
# tol_list = np.arange(-0.5,-0.1,0.02)
tol_list = np.arange(-0.5,-0.09,0.01)

def get_mean_and_std(tol_list,data_tuple_list_lc,data_tuple_list_lk):
    det_array_list = list()
    false_array_list = list()
    for tol in tol_list:
        det_array_list.append(getDetectionTimeArray(getDetectionTimeData(data_tuple_list_lc,tol)))
        false_times_array = getDetectionTimeData(data_tuple_list_lk,tol)
        false_array_list.append(np.array([len(array) for array in false_times_array]))

    det_time_mean_list = [det_array.mean() for det_array in det_array_list]
    det_time_std_list = [det_array.std() for det_array in det_array_list]
    false_mean_list = [false_array.mean() for false_array in false_array_list]
    false_std_list = [false_array.std() for false_array in false_array_list]
    return (det_time_mean_list,det_time_std_list,false_mean_list,false_std_list)

det_time_mean_list,det_time_std_list,false_mean_list,false_std_list = get_mean_and_std(tol_list,data_tuple_list_lc,data_tuple_list_lk)

fig = plt.figure()
ax1 = plt.subplot(2,1,1)
fig.set_size_inches(10,8)
plt.errorbar(1+np.array(tol_list),det_time_mean_list,yerr=det_time_std_list, fmt='o')
# plt.xlabel("Relative Likelyhood Threshold")
# plt.xlim((0.5,0.9))
plt.ylabel("Lane-change detection time (s)")

# fig_2 = plt.figure()
# fig_2.set_size_inches(10,4)
plt.subplot(2,1,2,sharex=ax1)
plt.errorbar(1+np.array(tol_list),false_mean_list,yerr=false_std_list,fmt='o')
plt.xlim((0.5,0.9))
plt.xlabel("Relative Likelyhood Threshold")
plt.ylabel("Avg. Num. of false detection")

# fig.savefig('false_detection_and_time_vs_threshold.pdf')

In [ ]:
tol_list = np.arange(-0.5,-0.09,0.01)
tol_list[10]

In [ ]:
# 2-axis plot
fig = plt.figure()
fig.set_size_inches(10,8)

color_list = ['r','g','b','c']
tol_list = np.arange(-0.5,-0.09,0.01)
for i,window_size in enumerate([5,10,15,20]):
    data_tuple_list_lc,data_tuple_list_lk = data_tuple_dict_winsize[window_size]
    det_time_mean_list,det_time_std_list,false_mean_list,false_std_list = get_mean_and_std(tol_list,data_tuple_list_lc,data_tuple_list_lk)
    plt.plot(false_mean_list,det_time_mean_list,'%so-'%(color_list[i]),label='Num of Observations: %s'%(window_size),linewidth=2.0)
    if True:
        ax = plt.gca()
#         print (false_mean_list[-1],det_time_mean_list[-1])
        if i > 0:
            ax.annotate('threshold = %s'%(1+tol_list[-1]), xy=(false_mean_list[-1],det_time_mean_list[-1]),  xycoords='data',
                        xytext=(0.25,1.5), textcoords='data',
                        bbox=dict(boxstyle="round", fc="0.8"),
                        arrowprops=dict(arrowstyle="->",shrinkA=0,shrinkB=10,connectionstyle="arc3,rad=0",color=color_list[i]))
#         print (false_mean_list[0],det_time_mean_list[0])
        ax.annotate('threshold = %s'%(1+tol_list[0]), xy=(false_mean_list[0],det_time_mean_list[0]),  xycoords='data',
                    xytext=(1,-3.75), textcoords='data',
                    bbox=dict(boxstyle="round", fc="0.8"),
                    arrowprops=dict(arrowstyle="->",shrinkA=0,shrinkB=10,connectionstyle="arc3,rad=0",color=color_list[i]))
        ax.annotate('threshold = %s'%(1+tol_list[10]), xy=(false_mean_list[10],det_time_mean_list[10]),  xycoords='data',
            xytext=(0.5,-1), textcoords='data',
            bbox=dict(boxstyle="round", fc="0.8"),
            arrowprops=dict(arrowstyle="->",shrinkA=0,shrinkB=10,connectionstyle="arc3,rad=0",color=color_list[i]))

plt.xlabel("Avg. num. of false detection (out of 500)")
plt.ylabel("Lane-change Detection Time (s)")
plt.xlim([-0.1,2.0])
plt.legend()

# fig.savefig('different_window_size.pdf')